# TFT Simulation

In [1]:
from params.default import *
md_code("params/default.py")

```python
# initialization


# environment needs to be set before hardware
# costs for environment
# is the cost of 1 kwh
simulation.environment.cost_power_kwh = "0.15 USD"
# is the cost of 1U rackspace per month in USD
simulation.environment.cost_rack_unit = "12 USD"

# sales parameters
# these are enduser prices for people buying capacity
# following prices are very aggressive
# compute unit = 4 GB memory and 2 virtual CPU, in market price between 40 and USD120
simulation.environment.sales_price_cu = "8 USD"
# storage unit = 1 TB of netto usable storage, in market prices between 20 and USD120
simulation.environment.sales_price_su = "5 USD"

# choose your hardware profile (other choices in stead of amd or supermicro or hpe)

# choose your token simulation !!!


# means at end of period we produce 40% more cpr (*1.4)
# cpr = cloud production rate (is like hashrate of bitcoin)
simulation.cpr_improve_set("0:0,60:40")

# price of a capacity unit goes down over time, here we say it will go down 40%
# means we expect price to be lowered by X e.g. 40 (/1.4)
simulation.cpr_sales_price_decline_set("0:0,60:40")

# utilization of the nodes, starting with 0, after 20 months starting March 2020 we go to 80% utilization
simulation.utilization_set("20:80,40:90")

# super important factor, how does token price goes up
# this is ofcourse complete speculation, no-one knows
simulation.tokenprice_set_5years(3)

##if you want to do it more specific
# 0:0 means month 0 we have value 0
# 60:3 means month 60: value is 3
# interpolation will happen between the values
# so below will let the price go from 0.15 first month to 3 over 60 months
# 119 is the last month we need to set (10 years from now, the relevant one is 5 years but simulator needs a further one
# simulation.tokenprice_set("0:0.15,60:3,119:6")

# month:growth_percent of nodes being added
# this means month 1 (is 0 in this file) we add 5 nodes, month 12 we add 1000 new nodes
simulation.nrnodes_new_set("0:5,6:150,12:1000,18:2000,24:8000,36:12000,48:20000,60:20000")
# simulation.nrnodes_new_set("0:5,6:150,20:5000")

# first batch of nodes added is 1500 nodes
# each node is +-4.5k usd (check the bill of material sheet)
# and we start the simulation with 800m tokens already farmed by the TF Farmers
simulation.nodesbatch_start_set(nrnodes=1500, months_left=36, tft_farmed_before_simulation=800 * 1000 * 1000)

simulation.calc()


```

In [2]:
md(f"""
## Simulation Arguments 

- nr of nodes at start: {simulation.nodebatches[0].nrnodes}
- nr of months left (avg out) for farming of startingnodes: {simulation.nodebatches[0].months_left}
- nr of TFT already farmed: { j.data.types.numeric.clean(simulation.nodebatches[0].tft_farmed_before_simulation)} 

""")


## Simulation Arguments 

- nr of nodes at start: 1500
- nr of months left (avg out) for farming of startingnodes: 36
- nr of TFT already farmed: 800,000,000 



## Device Details

In [3]:

md(f"""
### Device Details

- nr of TB netto usable = {round(server.su,1)} TB
- nr of cpu cores = {round(server.cru,1)} Cores
- power used = {server.power} watt
- performance in passmark = {round(server.cu_passmark*server.cu)}

""")


### Device Details

- nr of TB netto usable = 58.3 TB
- nr of cpu cores = 16.0 Cores
- power used = 232 watt
- performance in passmark = 24460



## Growth per month

In [4]:
x=np.arange(60)
fig = go.FigureWidget()
fig.add_trace(go.Scatter(x=x, y=simulation.sheet.rows.nrnodes_new.cells[0:60],name="new nodes"))
fig.add_trace(go.Scatter(x=x, y=simulation.rows.nrnodes_total.cells[0:60],name="total nr nodes"))
fig.update_layout(title = 'New/Total nr Nodes per Month',showlegend=True);

AttributeError: module 'plotly.graph_objects' has no attribute 'FigureWidget'

In [ ]:
fig

### Number of racks per month


In [ ]:
x=np.arange(0,60)
cells= [int(i/44) for i in simulation.sheet.rows.rackspace_u.cells[0:60]]
fig = go.FigureWidget(data=go.Scatter(x=x, y=cells))
fig.update_layout(title = 'Racks Occupied per Month',showlegend=False)
# fig

In [ ]:
cells= [int(i) for i in simulation.sheet.rows.power_kw.cells[0:60]]
fig = go.FigureWidget(data=go.Scatter(x=x, y=cells))
fig.update_layout(title = 'Kwatt/h per Month used.',showlegend=False)
fig

## TFT Growth

In [ ]:
names=["farmed","cultivated","sold","burned","farmer_income"]
def do(end):
    fig = go.FigureWidget()
    title = 'TFT Farmed, Cultivated, Sold, Burned per month, for batch month:%s.'%end
    xx=[i for i in range(1,end+1)]
    for i in range(5):
        name=names[i]
        cells=simulation.rows["tft_%s"%name].cells[1:end+1]
        fig.add_trace(go.Scatter(x=xx, y=cells,name=name))
    fig.update_layout(title = title,showlegend=True)
    return fig


In [ ]:
do(12)

In [ ]:
do(30)

In [ ]:
do(60)

In [ ]:
fig1, fig2, fig3, fig4, fig5, fig6, fig7,fig8=simulation.graph_tft_simulation(show=False);

In [ ]:
fig1

In [ ]:
fig2

In [ ]:
fig8

In [ ]:
fig3

In [ ]:
fig4


In [ ]:
fig5

In [ ]:
fig6

In [ ]:
fig7

## Cumulative TOKENS CREATED / BURNED / CULTIVATED / SOLD (in millions)


In [ ]:
m=1000000
simulation.rows.tft_sold.modify_invert()
simulation.rows.tft_burned.modify_invert()
rf=[int(i/m) for i in simulation.rows.tft_farmed.aggregate("Y",aggregate_type="SUM")]
rb=[int(i/m) for i in simulation.rows.tft_burned.aggregate("Y",aggregate_type="SUM")]
rs=[int(i/m) for i in simulation.rows.tft_sold.aggregate("Y",aggregate_type="SUM")]
rc=[int(i/m) for i in simulation.rows.tft_cultivated.aggregate("Y",aggregate_type="SUM")]

from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

labels=["tft_farmed","tft_burned","tft_sold","tft_cultivated"]
for year in range(2):
    data=[rf[year],rb[year],rs[year],rc[year]]
    graph = go.Pie(labels=labels, values=data, name="Year %s"%(year+1))
    fig.add_trace(graph,1,year+1)
fig.show()

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
y=1
for year in range(2,4):
    data=[rf[year],rb[year],rs[year],rc[year]]
    graph = go.Pie(labels=labels, values=data, name="Year %s"%(year+1))
    fig.add_trace(graph,1,y)
    y+=1
fig.show()
# md("### Details")



In [ ]:

fig = go.FigureWidget()
# fig.add_trace(go.Scatter(y=[2, 1, 4, 3]))
x=[1,2,3,4,5]
fig.add_trace(go.Bar(x=x,y=rf[0:5],name="Farming"))  #,text=["s sds s"]
fig.add_trace(go.Bar(x=x,y=rc[0:5],name="Cultivating"))
fig.add_trace(go.Bar(x=x,y=rs[0:5],name="Sold"))
fig.add_trace(go.Bar(x=x,y=rb[0:5],name="Burned"))
fig.update_layout(title = 'Farming / Cultivation / Burning Comparison',showlegend=True)
fig.show()

### Token price per month


In [ ]:
simulation.graph_token_price()


In [ ]:
#%qtconsole